In [4]:
!pip install pandas
import pandas as pd
import numpy as np
import requests # library to handle requests
import time
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

!conda install -c conda-forge folium --yes
import folium

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.



In [5]:
#Download store locations and remove unwanted columns

df = pd.read_csv("https://raw.githubusercontent.com/ninjananjo/Clothing-Store-Competition/master/StoreLocations.csv", encoding = 'ISO-8859-1')
df = df[['TOWN','NAME','ID','COUNTRY','Address1','Postcode','Host','PNO','NSF']]
#df = df[df['NSF'] > 1000] 
#df = df[(df['ID'].str.startswith("TS"))|(df['ID'].str.startswith("TM"))]
print(df.shape)
#df.head()

(387, 9)


In [6]:
#Group by Host

grouped_id = df.groupby('Host').apply(lambda x: ','.join(x.ID)).reset_index()
grouped_id.rename({0:'Branch IDs'}, axis='columns', inplace=True)
df = df.groupby(['Host']).last()
df = df.merge(grouped_id, left_on='Host',right_on='Host')
df.drop(columns=['ID'], inplace=True)
print(df.shape)
#df.head()

(204, 9)


In [7]:
#get latitude and longitude data

def getLocation(storeName, postcode, country):
    
    if country == 'ENG': country = "UK"
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(str(postcode)+" "+country, timeout=10)
    location_alt = geolocator.geocode(storeName+" "+country, timeout=10)
    try:
        lat = location.latitude
        lng = location.longitude
    except:
        try:
            lat = location_alt.latitude
            lng = location_alt.longitude
        except:
            lat = 0
            lng = 0
            
    return(lat,lng)

In [8]:
#Add latitude and longitude to dataframe

latitude = []
longitude = []

for storeName, postcode, country in zip(df['NAME'],df['Postcode'],df['COUNTRY']):
    lati, lngi = getLocation(storeName, postcode, country)
    latitude.append(lati)
    longitude.append(lngi)
    time.sleep(0.5)
    
df.loc[:,'Latitude']= latitude
df.loc[:,'Longitude']= longitude
#remove regions that location information was not available
df = df[(df['Latitude']!=0)| (df['Longitude']!=0)]
df.to_csv('StoreLocations_LatLng.csv', index=False)
print(df.shape[0])
df.head()

200


,Host,TOWN,NAME,COUNTRY,Address1,Postcode,PNO,NSF,Branch IDs,Latitude,Longitude
0,TM7157,OLDHAM,OLDHAM,ENG,Unit 3,OL1 1XE,1361,2200,TM7157,53.541672,-2.112246
1,TM7345,CHESTER,CHESTER,ENG,12/14 St Michael's Square,CH1 1EE,1748,3300,"TM7345,TS0324",53.189646,-2.889252
2,TM7439,Z EIRE DUBLIN,DUBLIN (GRAFTON ST),EIR,41 Grafton Street,NaN,5028,3300,TM7439,48.056514,-4.111926
3,TM7782,SHOREDITCH,SHOREDITCH,ENG,98 Commercial Street,E1 6LZ,968,1150,TM7782,51.520023,-0.074251
4,TM7815,DARLINGTON,DARLINGTON,ENG,28-34 Northgate,DL1 1NR,1660,2478,TM7815,54.526350,-1.554351


In [9]:
#Foursquare credentials

CLIENT_ID = 'QKP4KVQQZPVXMOKNIXHGQAT1E2IJZCHQPFB4XJGP3MC0CFUO' # your Foursquare ID
CLIENT_SECRET = '1N5BIRBR1EA5CO1QFMNHU1241SOFM5KJLLQRJGMDC0NOYWPH' # your Foursquare Secret
VERSION = '20190501' # Foursquare API version
LIMIT = 400 # limit of number of venues returned by Foursquare API
radius = 10000 # define radius
categoryId = '4bf58dd8d48988d1f2941735' #Sporting Goods Shop

In [10]:
#Find nearby sporting good shops

def getNearbyVenues(names, latitudes, longitudes, radius=500):    
    
    store_list = []
    lat_list = []
    lng_list = []
    
    for name, lat, lng in zip(names, latitudes, longitudes):
                    
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(    
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            categoryId)
        
        #print(url) 
        
        # make the GET request
        results = requests.get(url).json()["response"]
        
        #flatten JSON file
        resultsVen = json_normalize(results['venues'])
        resultsCat = json_normalize(data=results['venues'], record_path=['categories'], record_prefix='cat_', errors='ignore')
        resultsVenCat = pd.concat([resultsVen, resultsCat], axis=1)
        resultsVenCat = resultsVenCat[['id','name','location.distance','location.formattedAddress','location.lat','location.lng','cat_name']]
        
        #Add region name 
        for i in range(0,resultsVen.shape[0]):
            store_list.append(name)
            lat_list.append(lat)
            lng_list.append(lng)

        store = pd.DataFrame([store_list,lat_list,lng_list]).T
        nearbyVenues = pd.concat([store, resultsVenCat], axis=1)

    return(nearbyVenues)

In [11]:
regions_venues = pd.DataFrame()

#request venue information for each region and store in regions_venues
for storeId in df['Host'].unique():
    store_info = df[df['Host']==storeId]
    regions_venues = regions_venues.append(getNearbyVenues(names=store_info['Host'],
                                                  latitudes=store_info['Latitude'],
                                                  longitudes=store_info['Longitude'],
                                                  radius=radius),ignore_index=True)
    
#rename columns
regions_venues.rename(columns={0:'Host ID',
                          1:'Host Lat',
                          2:'Host Lng',
                          'id':'Venue ID',
                          'name':'Venue Name',
                          'location.distance':'Venue Distance',
                          'location.formattedAddress':'Venue Address',
                          'location.lat':'Venue Lat',
                          'location.lng':'Venue Lng',
                          'cat_name':'Venue Category'}, inplace=True)

print(regions_venues.shape[0])
regions_venues.head()

5478


,Host ID,Host Lat,Host Lng,Venue ID,Venue Name,Venue Distance,Venue Address,Venue Lat,Venue Lng,Venue Category
0,TM7157,53.5417,-2.11225,4bf93e604a67c928dd8026cf,JD Sports,132,[Units 16-17 Lower Mall The Spindles Sc (Lower...,53.541413,-2.114194,Sporting Goods Shop
1,TM7157,53.5417,-2.11225,5af008d0628c83002cdb1011,DW Fitness First,12659,"[Gelderd Road, Birstall, Leeds, WF17 9TD, Unit...",53.470197,-2.260968,Gym
2,TM7157,53.5417,-2.11225,5b7f0f5e4a7aae002cf533ea,JD Sports,7061,"[The Sharp Project (Thorpe Rd), Manchester, Gr...",53.501436,-2.194744,Sporting Goods Shop
3,TM7157,53.5417,-2.11225,4c0ab0ab340720a11ae88693,Umbro X Corrie,11866,[United Kingdom],53.477612,-2.255520,Sporting Goods Shop
4,TM7157,53.5417,-2.11225,5a3fe7778a6f1771f7fa9c3f,Patagonia,11016,"[Manchester, Greater Manchester, M2, United Ki...",53.481784,-2.244729,Sporting Goods Shop


In [12]:
#Excludes non JD Sports, adds host information then exports to csv

jdsports = regions_venues[regions_venues['Venue Name']=='JD Sports']
jdsports = jdsports.merge(df, left_on = 'Host ID', right_on = 'Host')
jdsports.drop(columns=['Host','Latitude','Longitude'], inplace=True)
column_order = jdsports.columns.tolist()
column_order = column_order[0:3] + column_order[10:] + column_order[3:10]
jdsports = jdsports[column_order]
jdsports.to_csv('jdsports_proximity.csv', index=False)
print(jdsports.shape[0])
jdsports.head()

505


,Host ID,Host Lat,Host Lng,TOWN,NAME,COUNTRY,Address1,Postcode,PNO,NSF,Branch IDs,Venue ID,Venue Name,Venue Distance,Venue Address,Venue Lat,Venue Lng,Venue Category
0,TM7157,53.5417,-2.11225,OLDHAM,OLDHAM,ENG,Unit 3,OL1 1XE,1361,2200,TM7157,4bf93e604a67c928dd8026cf,JD Sports,132,[Units 16-17 Lower Mall The Spindles Sc (Lower...,53.541413,-2.114194,Sporting Goods Shop
1,TM7157,53.5417,-2.11225,OLDHAM,OLDHAM,ENG,Unit 3,OL1 1XE,1361,2200,TM7157,5b7f0f5e4a7aae002cf533ea,JD Sports,7061,"[The Sharp Project (Thorpe Rd), Manchester, Gr...",53.501436,-2.194744,Sporting Goods Shop
2,TM7157,53.5417,-2.11225,OLDHAM,OLDHAM,ENG,Unit 3,OL1 1XE,1361,2200,TM7157,5a8ecd550881587526bae41b,JD Sports,9173,"[Unit 18 Crown Point North Denton, Denton, Tam...",53.459318,-2.117344,Sporting Goods Shop
3,TM7157,53.5417,-2.11225,OLDHAM,OLDHAM,ENG,Unit 3,OL1 1XE,1361,2200,TM7157,4c9dee1f46978cfa9f219c7f,JD Sports,5919,"[Units 28 & 29, The Arcades Shopping Centre (1...",53.489489,-2.095045,Sporting Goods Shop
4,TM7157,53.5417,-2.11225,OLDHAM,OLDHAM,ENG,Unit 3,OL1 1XE,1361,2200,TM7157,4c6aca2f897b1b8d72b1b217,JD Sports,2522,"[Unit 3 Centre Retail Park Elk Mill, Oldham, L...",53.558100,-2.138509,Sporting Goods Shop
